# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [538]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [539]:
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [540]:
df.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15157,1,-94.985702,47,среднее,1,женат / замужем,0,M,сотрудник,0,218320.752233,покупка недвижимости
16598,1,-3194.405293,34,высшее,0,в разводе,3,M,сотрудник,0,120044.814363,недвижимость
6947,0,-557.141552,39,среднее,1,Не женат / не замужем,4,F,сотрудник,0,56713.891275,недвижимость
1545,1,-901.467430,28,среднее,1,женат / замужем,0,M,сотрудник,0,103335.397996,заняться высшим образованием
3640,0,365664.390203,62,СРЕДНЕЕ,1,женат / замужем,0,M,пенсионер,0,364627.859998,сделка с подержанным автомобилем
18093,0,364183.024720,64,среднее,1,Не женат / не замужем,4,F,пенсионер,0,71498.487486,высшее образование
18430,1,-590.552843,32,неоконченное высшее,2,женат / замужем,0,F,компаньон,0,381383.996341,свой автомобиль
12750,0,-3564.768651,35,высшее,0,гражданский брак,1,F,сотрудник,0,374147.363604,сыграть свадьбу
10048,0,-8876.745097,43,ВЫСШЕЕ,0,женат / замужем,0,F,сотрудник,0,191108.362861,получение высшего образования
6418,0,364246.541992,53,среднее,1,женат / замужем,0,F,пенсионер,0,98114.342256,покупка коммерческой недвижимости


In [541]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [542]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [543]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


**Вывод**

1. Исходные данные имеют пропуски.
2. Замечены некорректные (отрицательные) данные в трудовом стаже.
3. Замечен разный стиль указания образования клиента.
4. Анамольные (единичные) значения в типах занятости клиентов. Значения в этом столбце не отражают сути его названия. Как я понимаю, тип занятости это полная/частичная 
5. Значения education_id, family_status_id на первый взгляд кажутся не иформативными. Правильные значения в education и family_status будут в полной мере заменять их.
6. Ежемесячный доход указал в float64. Слишком много значений после запятой, такая точность не требуется.
7. В целях получения кредита преобладает свадьба -> покупка жилья.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [544]:
df.loc[(df['days_employed'].isna()) & (df['total_income'].isna())]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Большое кол-во сотрудников не имеют трудовой стаж в днях. Заполнять пропуски средними значениями на данном этапе нет смысла, т.к. присутствуют много отрицательных значений и такие варинта когда стаж работника большое его возраста.

2.1.1 Заполнение пропусков в df['days_employed'] (общий трудовой стаж в днях) 

In [545]:
df.loc[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


In [546]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean().astype(int)

In [547]:
df.loc[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [548]:
def employed_coef(row):
    if row['days_employed'] != 0:
        return abs(row['days_employed']/(365*row['dob_years']))
    return 0

In [549]:
df['employed_coef'] = df.apply(employed_coef, axis=1)

In [550]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.550403
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.306302
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.466868
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.353146
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,17.589355
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,0.093981
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,0.183447
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.008371
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,0.542455
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,0.146258


In [551]:
df.loc[df['employed_coef'] > 1]['employed_coef'].count()

3445

In [552]:
employed_coef = df.loc[df['employed_coef'] < 1]['employed_coef'].mean()
employed_coef

0.15780152801718478

In [553]:
df['days_employed'] = df['days_employed'].fillna(df['dob_years']*365*employed_coef)

In [554]:
average_income = df['total_income'].median()
average_income

145017.93753253992

In [555]:
df['total_income'] = df['total_income'].fillna(average_income)

In [556]:
df['employed_coef'] = df['employed_coef'].fillna(df['days_employed']/(365*df['dob_years']))

In [557]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.550403
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.306302
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.466868
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.353146
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,17.589355
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,0.093981
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,0.183447
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.008371
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,0.542455
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,0.146258


In [558]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
employed_coef       21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


In [559]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,21525.0,56933.845163,134765.896436,-18388.949901,-2518.168900,-982.531720,1612.731616,4.017554e+05
dob_years,21525.0,43.495145,12.218213,19.000000,34.000000,43.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,21525.0,165159.487397,97866.068069,20667.263793,107798.172619,145017.937533,195543.620942,2.265604e+06
employed_coef,21525.0,2.867256,6.285852,0.001760,0.076351,0.157802,0.319510,4.168920e+01


**Вывод**

Все пропуски в исходной таблице были связанны с количественными данными: доход и трудовой стаж клиентов. Такие пропуски могут появиться в случае отсутсвия у клиента официального места работы. Был сделан вывод, что у всех клиентов не имеющих трудового стажа, заменить пропуски на нулевые значения. Сделано это и потому что в этом столбце явные проблемы с данными (много отрицательных значений, есть случаи в которых стаж превышает возраст клиента) и применять средние значения не корректно. Пропуски в месечной доходности заменены на средние доходности по таблице.

### Обработка дубликатов

2.2.0 Замена вещественного типа данных на целочисленный

In [560]:
df = df.astype({'days_employed': 'int32', 'total_income': 'int32', 'children': 'int8', 'dob_years': 'int8',
           'education_id': 'int8', 'family_status_id': 'int8', 'employed_coef': 'float16', 'debt': 'int8'})

In [561]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
employed_coef       21525 non-null float16
dtypes: float16(1), int32(2), int8(5), object(5)
memory usage: 1.1+ MB


Исправление отрицательных значений в days_employed (общий трудовой стаж в днях) путем взятия его по модулю

In [562]:
df.sample(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef
632,0,-3733,39,среднее,1,женат / замужем,0,M,компаньон,0,188899,покупка жилья для сдачи,0.262207
19310,3,-1029,32,высшее,0,гражданский брак,1,F,компаньон,0,155791,на проведение свадьбы,0.088135
20260,0,-221,44,среднее,1,женат / замужем,0,F,госслужащий,0,187570,строительство недвижимости,0.013794
19205,2,-3517,32,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,82428,покупка жилья,0.301025
12953,1,346331,59,среднее,1,женат / замужем,0,F,пенсионер,0,153731,строительство жилой недвижимости,16.078125
9558,2,2188,38,среднее,1,женат / замужем,0,M,компаньон,0,145017,строительство собственной недвижимости,0.157837
13701,0,-429,51,высшее,0,женат / замужем,0,F,компаньон,0,195993,операции со своей недвижимостью,0.023071
262,0,-1617,44,среднее,1,женат / замужем,0,F,сотрудник,0,168586,покупка недвижимости,0.100708
12266,1,1497,26,среднее,1,женат / замужем,0,M,сотрудник,0,145017,покупка жилья для семьи,0.157837
16311,0,-1456,26,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,185945,покупка коммерческой недвижимости,0.153442


Почему возникли такие данные в этом столбце? Возможно из-за неправильного алгоритма расчета. К примеру были перепутаны местами дата начала и дата окончания работы.

In [563]:
def to_days(row):
    day_emplyed = row['days_employed']
    employed_coef = row['employed_coef']
    if day_emplyed < 0:
        day_emplyed = abs(day_emplyed)
    if employed_coef > 1:
        return day_emplyed/24
    return day_emplyed

In [564]:
df['days_employed'] = df.apply(to_days, axis=1)

In [565]:
df = df.astype({'days_employed': 'int16'})

In [566]:
df['children'] = df['children'].abs()

In [567]:
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Аномалия в кол-ве детей (20). Возможно удалить эти записи или предположить что это ошибочная запись числа два. Ниже будет добавлен обобщенный индикатор для этого параментра.

In [568]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0.550293
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.306396
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0.466797
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0.353027
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,17.593750
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0.093994
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0.183472
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,0.008369
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0.542480
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0.146240


2.2.1 Поиск дубликатов

In [569]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В столбце "уровень образования клиента" много дубликтов из за разного регистра, которые нужно исправить. Такое возможно возникало из за разных стандартов заполнения данных в разных филиалах банка. Возможно это было не жестко регламентировано для сотрудника, который заполнял данных о клиенте.

In [570]:
df['education'] = df['education'].str.lower()

In [571]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Дубликаты в уровне образования устранены

In [572]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [573]:
df['family_status'] = df['family_status'].str.lower()

In [574]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Устранил разный регистр. В этом столбце дубликаты не обнаружены.

In [575]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

В столбце "тип занятости" (income_type) очень узкие категории клиентов в которых мало данных. Ими можно пожертвовать и удалить ради более чистых данных.
Почему?
Их общее кол-во сильно меньше 1% от всех данных таблицы.

In [576]:
df.drop([*(df[df['income_type'] == 'предприниматель'].index.to_list()), 
         *(df[df['income_type'] == 'безработный'].index.to_list()), 
         *(df[df['income_type'] == 'студент'].index.to_list()), 
         *(df[df['income_type'] == 'в декрете'].index.to_list())], inplace=True)

In [577]:
df['income_type'].value_counts()

сотрудник      11119
компаньон       5085
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64

In [578]:
df['income_type'].value_counts()

сотрудник      11119
компаньон       5085
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64

In [579]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0.550293
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.306396
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0.466797
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0.353027
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,17.593750
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0.093994
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0.183472
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0.008369
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0.542480
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0.146240


**Вывод**

Была проведена оценка каждого столбца с данными методом value_counts(). Были выявлены дубликаты и предприняты меры по их устранению.

1. Большенство дубликатов в данных связано в заполнении данных разным регистром. Это легко исправляется приведением всех записей к общему виду.
2. В столбце df['income_type'] было несколько узких категорий и 1=2 записями в них. Таким кол-вом данных можно пожертвовать безболезненно для частоты анализа.

### Лемматизация

In [580]:
m = Mystem() 

def lemmatize_text(text):
    return m.lemmatize(text)

df['purpose_lemmatized'] = df['purpose'].apply(lemmatize_text)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef,purpose_lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0.550293,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.306396,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0.466797,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0.353027,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,17.593750,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,0.288574,"[операция, , с, , жилье, \n]"
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,14.062500,"[сделка, , с, , автомобиль, \n]"
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,0.152344,"[недвижимость, \n]"
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,0.224365,"[на, , покупка, , свой, , автомобиль, \n]"


In [581]:
def counter_data_lemmas(col):
    list_data = []
    for c in col:
        for word in c:
            if word not in [' ', '\n']:
                list_data.append(word)
    return list_data

Counter(counter_data_lemmas(df['purpose_lemmatized']))

Counter({'покупка': 5910,
         'жилье': 4471,
         'приобретение': 462,
         'автомобиль': 4314,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2347,
         'операция': 2610,
         'с': 2924,
         'на': 2232,
         'проведение': 776,
         'для': 1293,
         'семья': 641,
         'недвижимость': 6365,
         'коммерческий': 1315,
         'жилой': 1232,
         'строительство': 1880,
         'собственный': 634,
         'подержать': 479,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'подержанный': 489,
         'получение': 1316,
         'высокий': 1375,
         'профильный': 436,
         'сдача': 652,
         'ремонт': 611})

**Вывод**

Была проведена лемматизация целей кредита у клиентов с помощью инструментов библиотеки pymystem3. В процессе изучения результатов были обнаружены частые повторения пустых строк и символов переноса строки. Такие данные были изключениы из итогового подсчета лемм.
Было обнаружено, что самыми популярными целями взятия кредитов у клиентов является недвижимость.

### Категоризация данных

In [582]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef,purpose_lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0.550293,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.306396,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0.466797,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0.353027,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,17.593750,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0.093994,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0.183472,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0.008369,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0.542480,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0.146240,"[покупка, , жилье, , для, , семья, \n]"


Создадим сводную таблицу, которое покажет влияние семейного положения на на факт погашения кредита в срок:

Обобщенная функция создания сводных таблиц по названию столбца:

In [583]:
df[df['family_status'] == 'женат / замужем']['debt'].value_counts()

0    11448
1      929
Name: debt, dtype: int64

In [584]:
def calc_debt_pct_by_title(title):
    pct_data = []
    title_list = df[title].unique()
    for t in title_list:
        frame = df[(df[title] == t)]['debt'].value_counts()
        pct = frame.loc[0] / (frame.loc[0] + frame.loc[1]) * 100
        pct_data.append(pct)
    data_frame = pd.DataFrame({title: title_list.tolist(), 'pct': pct_data})
    return data_frame

Сводная таблица по семейному статусу:

In [585]:
family_status = calc_debt_pct_by_title('family_status')
family_status.sort_values(by='pct', ascending=False)

,family_status,pct
2,вдовец / вдова,93.437500
3,в разводе,92.887029
0,женат / замужем,92.494142
1,гражданский брак,90.706587
4,не женат / не замужем,90.256046


Обобщенная категоризация по кол-ву детей:

In [586]:
def is_children(row):
    if row['children'] > 2:
        return 'более 2-х детей'
    if row['children'] > 0:
        return '1-2 ребенка'
    return 'нет детей'

In [587]:
df['is_children'] = df.apply(is_children, axis=1)

In [588]:
children_status = calc_debt_pct_by_title('is_children')
children_status.sort_values(by='pct', ascending=False)

,is_children,pct
1,нет детей,92.484977
2,более 2-х детей,91.447368
0,1-2 ребенка,90.792136


Обобщенная категоризация по ежемесячному доходу:

In [589]:
def income_lvl(row):
    if row['total_income'] >= 300000:
        return 'более 300 тыс. руб.'
    if 299999 > row['total_income'] >= 200000:
        return 'от 200 до 300 тыс. руб'
    if 199999 > row['total_income'] >= 100000:
        return 'от 100 до 200 тыс. руб'
    return 'менее 100 тыс.руб'

In [590]:
df['income_lvl'] = df.apply(income_lvl, axis=1)

In [591]:
df['income_lvl'].value_counts()

от 100 до 200 тыс. руб    11995
менее 100 тыс.руб          4460
от 200 до 300 тыс. руб     3583
более 300 тыс. руб.        1481
Name: income_lvl, dtype: int64

Сводная таблица:

In [592]:
income_status = calc_debt_pct_by_title('income_lvl')
income_status.sort_values(by='pct', ascending=False)

,income_lvl,pct
0,от 200 до 300 тыс. руб,92.966788
3,более 300 тыс. руб.,92.842674
2,менее 100 тыс.руб,92.107623
1,от 100 до 200 тыс. руб,91.421426


In [593]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_coef,purpose_lemmatized,is_children,income_lvl
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0.550293,"[покупка, , жилье, \n]",1-2 ребенка,от 200 до 300 тыс. руб
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0.306396,"[приобретение, , автомобиль, \n]",1-2 ребенка,от 100 до 200 тыс. руб
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0.466797,"[покупка, , жилье, \n]",нет детей,от 100 до 200 тыс. руб
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,0.353027,"[дополнительный, , образование, \n]",более 2-х детей,от 200 до 300 тыс. руб
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,17.593750,"[сыграть, , свадьба, \n]",нет детей,от 100 до 200 тыс. руб


In [594]:
def credit_type(row):
    for r in row['purpose_lemmatized']:
        if r in ['жилье', 'недвижимость', 'строительство']:
            return 'недвижимость'
        elif r == 'авто':
            return 'авто'
        elif r == 'образование':
            return 'образование'
        elif r == 'свадьба':
            return 'свадьба'
    return 'другое'

In [595]:
df['credit_type'] = df.apply(credit_type, axis=1)

In [596]:
df['credit_type'].value_counts()

недвижимость    10836
другое           4314
образование      4022
свадьба          2347
Name: credit_type, dtype: int64

In [597]:
credit_type_status = calc_debt_pct_by_title('credit_type')
credit_type_status.sort_values(by='pct', ascending=False)

,credit_type,pct
0,недвижимость,92.792543
3,свадьба,92.074989
2,образование,90.800597
1,другое,90.681502


**Вывод**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [598]:
children_status.sort_values(by='pct', ascending=False)

,is_children,pct
1,нет детей,92.484977
2,более 2-х детей,91.447368
0,1-2 ребенка,90.792136


**Вывод**

В самом распространенном случае (1-2 ребенка) процент возврата кредитов самый маленький. Отсутствие детей благоприятно влияет на возвраты кредитов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [599]:
family_status.sort_values(by='pct', ascending=False)

,family_status,pct
2,вдовец / вдова,93.437500
3,в разводе,92.887029
0,женат / замужем,92.494142
1,гражданский брак,90.706587
4,не женат / не замужем,90.256046


**Вывод**

Не женат / не замужем можно рассматривать как самую низкоответсвенную категорию из всех. Соответственно процент возврата займов в этой категории самый низкий, что логично.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [600]:
income_status.sort_values(by='pct', ascending=False)

,income_lvl,pct
0,от 200 до 300 тыс. руб,92.966788
3,более 300 тыс. руб.,92.842674
2,менее 100 тыс.руб,92.107623
1,от 100 до 200 тыс. руб,91.421426


**Вывод**

Уровень дохода слабо влияет на процент возврата кредитов. Более высокий доход не равен более благоприятному проценту возврата.

- Как разные цели кредита влияют на его возврат в срок?

In [601]:
credit_type_status = calc_debt_pct_by_title('credit_type')
credit_type_status.sort_values(by='pct', ascending=False)

,credit_type,pct
0,недвижимость,92.792543
3,свадьба,92.074989
2,образование,90.800597
1,другое,90.681502


**Вывод**

У недвижимости значиетельно лучше процент возвратов кредитов, нежели у образования и прочих целей

## Шаг 4. Общий вывод

В ходе ииследования предоставленной мне базы клиентов банка, мною были сделаны следщие выводы:

В реальных данных очень часто встечаются значения, которые не отражают действительность, так называемые артефакты. Их поиск, обнаружение и исправления важны не только для анализа полученых данных, но и для предупреждения будущих неправильных исследований. От выбора правильного способа устанения таких артефатов, зависит чистота и правильность итового анализа.

Были сделанны нектороые преположения о влиянии ряда параметров клиента на его собосность возвращать кредитные задолжности в срок.

- Можно сделать вывод что в худшем случае у каждого десятого клиент имеются проблемы с возвратом кредита в срок.
- Есть ли зависимость между наличием детей и возвратом кредита в срок? Появление ребенка некативно влиияет на способность клиентов вовращать кредитные задолженности. падает с 92.48% до 90.79%. Такая статистика справедлива для клиентов, имеющих 1-2 ребенка в семье, что является наиболее частым случаем кол-ва детей. Но расет с появлением 3 и более ребенка до 91.44%.

- Семейное положение также влияет на возможности клиентов возврата кредитов. У людей без официальной социальной ответсвенноти она ниже (не женат / не замужем:90.25%) в то время как (вдовец / вдова:93.43%). Клиенты во время и после официальных семейных взаимоотношений относятся к кредитным задолженностям более отвественно нежелю люди без отношений или живущих в гражанском браке.

- Не было замечено сильной зависимости ежемесячного дохода от факта уплаты кредитов. Показатели колеблются от 91.42% до 92.96%. Возможно это связано с тем, что люди с большим достатком берут большие кредитные займы, и наоборот.

- Что касается целей кредитов, самыми надежными, по возврату оказались кредиты на недвижимость	92.79%, а самые неблагополучные - образование 90.80% 

Полученные выводы могут помочь банку в принятии решений о выдаче очередного кредита для клиента. Правильные решения повысят общий процент своевременных возвратов.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.